at first make sure to run app.py. In this regard you should go to the app.py repository and run "flask run" command in your terminal. If you install all the requirment correctly, you should see in your command line that the flask is running in this server: Running on http://127.0.0.1:5000
The server and the port should be the same otherwise you should change the code below.

In [1]:
from serving.serving_client import ServingClient

In [2]:
from game.game_client import GameClient

In [3]:
gc = GameClient()

In [4]:
#get data

In [70]:
df = gc.ping_game(2022020530)

/home/nexus10/Documents/UoMontreal_2022/IFT6758 Data Science/IFT6758-docker-project/ift6758/ift6758/client/game/game_client.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.game_list[game_id].append(df)


In [59]:
#predict

In [64]:
df.head()

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
0,6,01:19,1,2022020530,Seattle Kraken,Vancouver Canucks,0,65.0,-13.0,4,...,5,5,0,27.2947,53.366656,0.4964,1.2723,79,0,0
1,12,01:40,1,2022020530,Seattle Kraken,Vancouver Canucks,0,56.0,12.0,4,...,5,5,0,35.1141,16.401219,0.3488,0.8330,40,0,0
2,16,02:54,1,2022020530,Seattle Kraken,Vancouver Canucks,0,66.0,-13.0,4,...,5,5,0,26.4197,49.648766,0.5145,0.8599,54,0,0
3,30,05:47,1,2022020530,Seattle Kraken,Vancouver Canucks,0,62.0,-17.0,4,...,5,5,0,31.9061,NaN,0.5619,NaN,47,0,0
4,38,07:01,1,2022020530,Seattle Kraken,Vancouver Canucks,1,24.0,-39.0,2,...,5,5,0,75.8024,77.103826,0.5404,0.4429,61,0,0


In [61]:
df = df.dropna()

In [ ]:
# from serving_clinet import ServingClient

In [9]:
sc= ServingClient()

In [10]:
sc.download_registry_model(workspace='ift-6758-2', model='xgb-model-5-2-pickle', version='1.0.0', model_name='model_5_2.pickle')


In [12]:
for i in df['event_idx']:
    a = df.loc[df['event_idx'] == i]
    d = sc.predict(a)
    print(i)    
    print(d)

4
[0.0120560796931386]
8
[0.013232256285846233]
16
[0.015472229570150375]
20
[0.03968000039458275]
27
[0.010195557028055191]
34
[0.05021725222468376]
40
[0.07908698171377182]
41
[0.13184553384780884]
44
[0.12061066925525665]
50
[0.012955030426383018]
53
[0.10778184235095978]
59
[0.22497136890888214]
62
[0.21449653804302216]
63
[0.07944712042808533]
65
[0.22535140812397003]
66
[0.03608245030045509]
78
[0.09222190827131271]
79
[0.1788523644208908]
80
[0.3284715414047241]
91
[0.18140508234500885]
96
[0.0164080411195755]
99
[0.17376260459423065]
106
[0.2288747876882553]
127
[0.22391805052757263]
128
[0.03623928502202034]
129
[0.07272914052009583]
130
[0.0968940407037735]
132
[0.10847863554954529]
135
[0.06003790721297264]
137
[0.1541789025068283]
149
[0.038206350058317184]
151
[0.14254385232925415]
155
[0.048448771238327026]
156
[0.10830796509981155]
157
[0.10876937955617905]
172
[0.08135280758142471]
176
[0.1051832064986229]
179
[0.027238281443715096]
182
[0.016368893906474113]
195
[0.051

In [30]:
a = df.loc[df['event_idx'] == '210']

In [31]:
a

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
43,210,05:31,3,2021020328,New York Islanders,New York Rangers,0,-66.0,-25.0,3,...,5,5,0,33.9706,NaN,0.827,NaN,31,0,0


In [35]:
d = sc.predict(df)

In [36]:
print(d)

[0.0120560796931386, 0.013232256285846233, 0.015472229570150375, 0.03968000039458275, 0.010195557028055191, 0.05021725222468376, 0.07908698171377182, 0.13184553384780884, 0.12061066925525665, 0.012955030426383018, 0.10778184235095978, 0.22497136890888214, 0.21449653804302216, 0.07944712042808533, 0.22535140812397003, 0.03608245030045509, 0.09222190827131271, 0.1788523644208908, 0.3284715414047241, 0.18140508234500885, 0.0164080411195755, 0.17376260459423065, 0.2288747876882553, 0.22391805052757263, 0.03623928502202034, 0.07272914052009583, 0.0968940407037735, 0.10847863554954529, 0.06003790721297264, 0.1541789025068283, 0.038206350058317184, 0.14254385232925415, 0.048448771238327026, 0.10830796509981155, 0.10876937955617905, 0.08135280758142471, 0.1051832064986229, 0.027238281443715096, 0.016368893906474113, 0.051382046192884445, 0.053008802235126495, 0.04321276769042015, 0.03678126260638237, 0.35954704880714417, 0.05465275049209595, 0.09703370183706284, 0.02969271130859852, 0.20535534

In [39]:
df['xg'] = d

/tmp/ipykernel_3278/1314773963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xg'] = d


In [71]:
df

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
0,6,01:19,1,2022020530,Seattle Kraken,Vancouver Canucks,0,65.0,-13.0,4,...,5,5,0,27.2947,53.366656,0.4964,1.2723,79,0,0
1,12,01:40,1,2022020530,Seattle Kraken,Vancouver Canucks,0,56.0,12.0,4,...,5,5,0,35.1141,16.401219,0.3488,0.8330,40,0,0
2,16,02:54,1,2022020530,Seattle Kraken,Vancouver Canucks,0,66.0,-13.0,4,...,5,5,0,26.4197,49.648766,0.5145,0.8599,54,0,0
3,30,05:47,1,2022020530,Seattle Kraken,Vancouver Canucks,0,62.0,-17.0,4,...,5,5,0,31.9061,NaN,0.5619,NaN,47,0,0
4,38,07:01,1,2022020530,Seattle Kraken,Vancouver Canucks,1,24.0,-39.0,2,...,5,5,0,75.8024,77.103826,0.5404,0.4429,61,0,0
5,46,08:27,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-33.0,-36.0,1,...,5,5,0,66.5733,23.194827,0.5713,0.4586,27,0,0
6,47,09:09,1,2022020530,Seattle Kraken,Vancouver Canucks,0,85.0,22.0,4,...,5,5,0,22.3607,131.483839,1.3909,0.5713,69,0,0
7,49,09:45,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-71.0,4.0,4,...,5,5,0,18.4391,NaN,0.2187,NaN,45,0,0
8,61,10:59,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-54.0,-19.0,4,...,5,5,0,39.8246,129.653384,0.4973,0.8330,59,0,0
9,68,12:08,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-61.0,-18.0,4,...,5,4,48,33.2866,57.271284,0.5713,1.4537,8,0,0


In [ ]:
 home_xG_s = df.loc[df['team_name'] == 'Washington Capitals']

In [ ]:
home_xG_s['xg'].sum()

In [13]:
import requests
import json

In [17]:
r = requests.post(
"http://127.0.0.1:5000/predict",
json=json.loads(df.to_json())
)
print(r.json())

[0.0120560796931386, 0.013232256285846233, 0.015472229570150375, 0.03968000039458275, 0.010195557028055191, 0.05021725222468376, 0.07908698171377182, 0.13184553384780884, 0.12061066925525665, 0.012955030426383018, 0.10778184235095978, 0.22497136890888214, 0.21449653804302216, 0.07944712042808533, 0.22535140812397003, 0.03608245030045509, 0.09222190827131271, 0.1788523644208908, 0.3284715414047241, 0.18140508234500885, 0.0164080411195755, 0.17376260459423065, 0.2288747876882553, 0.22391805052757263, 0.03623928502202034, 0.07272914052009583, 0.0968940407037735, 0.10847863554954529, 0.06003790721297264, 0.1541789025068283, 0.038206350058317184, 0.14254385232925415, 0.048448771238327026, 0.10830796509981155, 0.10876937955617905, 0.08135280758142471, 0.1051832064986229, 0.027238281443715096, 0.016368893906474113, 0.051382046192884445, 0.053008802235126495, 0.04321276769042015, 0.03678126260638237, 0.35954704880714417, 0.05465275049209595, 0.09703370183706284, 0.02969271130859852, 0.20535534

In [75]:
df_1 = df.iloc[:40,:]
df_2 = df.iloc[40:,:]

In [76]:
df_1

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
0,6,01:19,1,2022020530,Seattle Kraken,Vancouver Canucks,0,65.0,-13.0,4,...,5,5,0,27.2947,53.366656,0.4964,1.2723,79,0,0
1,12,01:40,1,2022020530,Seattle Kraken,Vancouver Canucks,0,56.0,12.0,4,...,5,5,0,35.1141,16.401219,0.3488,0.8330,40,0,0
2,16,02:54,1,2022020530,Seattle Kraken,Vancouver Canucks,0,66.0,-13.0,4,...,5,5,0,26.4197,49.648766,0.5145,0.8599,54,0,0
3,30,05:47,1,2022020530,Seattle Kraken,Vancouver Canucks,0,62.0,-17.0,4,...,5,5,0,31.9061,NaN,0.5619,NaN,47,0,0
4,38,07:01,1,2022020530,Seattle Kraken,Vancouver Canucks,1,24.0,-39.0,2,...,5,5,0,75.8024,77.103826,0.5404,0.4429,61,0,0
5,46,08:27,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-33.0,-36.0,1,...,5,5,0,66.5733,23.194827,0.5713,0.4586,27,0,0
6,47,09:09,1,2022020530,Seattle Kraken,Vancouver Canucks,0,85.0,22.0,4,...,5,5,0,22.3607,131.483839,1.3909,0.5713,69,0,0
7,49,09:45,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-71.0,4.0,4,...,5,5,0,18.4391,NaN,0.2187,NaN,45,0,0
8,61,10:59,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-54.0,-19.0,4,...,5,5,0,39.8246,129.653384,0.4973,0.8330,59,0,0
9,68,12:08,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-61.0,-18.0,4,...,5,4,48,33.2866,57.271284,0.5713,1.4537,8,0,0


In [77]:
df_2

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
40,185,14:08,2,2022020530,Seattle Kraken,Vancouver Canucks,0,43.0,14.0,4,...,5,5,0,48.0833,NaN,0.2954,NaN,8,0,0
41,190,15:25,2,2022020530,Seattle Kraken,Vancouver Canucks,0,87.0,-9.0,3,...,5,5,0,9.2195,38.013156,1.3521,0.2450,85,0,0
42,191,15:29,2,2022020530,Seattle Kraken,Vancouver Canucks,0,39.0,27.0,1,...,5,5,0,56.8243,60.000000,0.4951,1.3521,29,0,0


In [79]:
if df_1.iloc[-1, df.columns.get_loc('event_idx')] == df.iloc[-1, df.columns.get_loc('event_idx')]:
    print("Same Game")
    print(df_1.iloc[-1, df_1.columns.get_loc('event_idx')])
    print(df.iloc[-1, df.columns.get_loc('event_idx')])
else:
    print("different game")
    a = df_1.iloc[-1, df_1.columns.get_loc('event_idx')]
    print(a)

different game
183


In [80]:
a = str(a)

In [81]:
b = df[df['event_idx']==a].index.values

In [82]:
b[0]

39

In [85]:
df_3 = df.iloc[b[0]+1:,:]

In [87]:
df_3

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
40,185,14:08,2,2022020530,Seattle Kraken,Vancouver Canucks,0,43.0,14.0,4,...,5,5,0,48.0833,NaN,0.2954,NaN,8,0,0
41,190,15:25,2,2022020530,Seattle Kraken,Vancouver Canucks,0,87.0,-9.0,3,...,5,5,0,9.2195,38.013156,1.3521,0.2450,85,0,0
42,191,15:29,2,2022020530,Seattle Kraken,Vancouver Canucks,0,39.0,27.0,1,...,5,5,0,56.8243,60.000000,0.4951,1.3521,29,0,0


In [88]:
import pandas as pd

In [89]:
df_4 = pd.concat([df_1,df_3])

In [90]:
df_4

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
0,6,01:19,1,2022020530,Seattle Kraken,Vancouver Canucks,0,65.0,-13.0,4,...,5,5,0,27.2947,53.366656,0.4964,1.2723,79,0,0
1,12,01:40,1,2022020530,Seattle Kraken,Vancouver Canucks,0,56.0,12.0,4,...,5,5,0,35.1141,16.401219,0.3488,0.8330,40,0,0
2,16,02:54,1,2022020530,Seattle Kraken,Vancouver Canucks,0,66.0,-13.0,4,...,5,5,0,26.4197,49.648766,0.5145,0.8599,54,0,0
3,30,05:47,1,2022020530,Seattle Kraken,Vancouver Canucks,0,62.0,-17.0,4,...,5,5,0,31.9061,NaN,0.5619,NaN,47,0,0
4,38,07:01,1,2022020530,Seattle Kraken,Vancouver Canucks,1,24.0,-39.0,2,...,5,5,0,75.8024,77.103826,0.5404,0.4429,61,0,0
5,46,08:27,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-33.0,-36.0,1,...,5,5,0,66.5733,23.194827,0.5713,0.4586,27,0,0
6,47,09:09,1,2022020530,Seattle Kraken,Vancouver Canucks,0,85.0,22.0,4,...,5,5,0,22.3607,131.483839,1.3909,0.5713,69,0,0
7,49,09:45,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-71.0,4.0,4,...,5,5,0,18.4391,NaN,0.2187,NaN,45,0,0
8,61,10:59,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-54.0,-19.0,4,...,5,5,0,39.8246,129.653384,0.4973,0.8330,59,0,0
9,68,12:08,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-61.0,-18.0,4,...,5,4,48,33.2866,57.271284,0.5713,1.4537,8,0,0


In [54]:
p = len(df.loc[df['team_name'] == 'New York Islanders'])

In [55]:
p

28

In [91]:
df

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
0,6,01:19,1,2022020530,Seattle Kraken,Vancouver Canucks,0,65.0,-13.0,4,...,5,5,0,27.2947,53.366656,0.4964,1.2723,79,0,0
1,12,01:40,1,2022020530,Seattle Kraken,Vancouver Canucks,0,56.0,12.0,4,...,5,5,0,35.1141,16.401219,0.3488,0.8330,40,0,0
2,16,02:54,1,2022020530,Seattle Kraken,Vancouver Canucks,0,66.0,-13.0,4,...,5,5,0,26.4197,49.648766,0.5145,0.8599,54,0,0
3,30,05:47,1,2022020530,Seattle Kraken,Vancouver Canucks,0,62.0,-17.0,4,...,5,5,0,31.9061,NaN,0.5619,NaN,47,0,0
4,38,07:01,1,2022020530,Seattle Kraken,Vancouver Canucks,1,24.0,-39.0,2,...,5,5,0,75.8024,77.103826,0.5404,0.4429,61,0,0
5,46,08:27,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-33.0,-36.0,1,...,5,5,0,66.5733,23.194827,0.5713,0.4586,27,0,0
6,47,09:09,1,2022020530,Seattle Kraken,Vancouver Canucks,0,85.0,22.0,4,...,5,5,0,22.3607,131.483839,1.3909,0.5713,69,0,0
7,49,09:45,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-71.0,4.0,4,...,5,5,0,18.4391,NaN,0.2187,NaN,45,0,0
8,61,10:59,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-54.0,-19.0,4,...,5,5,0,39.8246,129.653384,0.4973,0.8330,59,0,0
9,68,12:08,1,2022020530,Seattle Kraken,Vancouver Canucks,0,-61.0,-18.0,4,...,5,4,48,33.2866,57.271284,0.5713,1.4537,8,0,0
